In [ ]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from sklearn.preprocessing import StandardScaler

import seaborn as sns

In [ ]:
def automate_data_processing(df, target_column, missing_value_threshold=0.5):
    threshold = len(df) * missing_value_threshold
    df_cleaned = df.dropna(thresh=threshold, axis=1)
    
    df_cleaned = df_cleaned.dropna(subset=[target_column])
    df_cleaned['main_category'] = df['main_category']
    y = df_cleaned[target_column]
    X = df_cleaned.drop(columns=[target_column])

    df_cleaned = df_cleaned.drop_duplicates()
    
    return df_cleaned, X, y

def clean_outliers(df):
    cols_100g = [col for col in df.columns if col.endswith('_100g')]
    
    df[cols_100g] = df[cols_100g].applymap(lambda x: abs(x) if pd.notnull(x) else x)

    if 'energy_100g' in df.columns:
        df['energy_100g'] = df['energy_100g'].apply(lambda x: x if x <= 5000 else float('NaN'))
    
    for col in cols_100g:
        if col != 'energy_100g':
            df[col] = df[col].apply(lambda x: x if x <= 100 else float('NaN'))

    return df

def clean_nutrient_outliers(df):
    # Définir les seuils pour chaque nutriment
    nutrient_thresholds = {
        'fat_100g': 100,
        'saturated-fat_100g': 80,
        'carbohydrates_100g': 100,
        'sugars_100g': 100,
        'fiber_100g': 30,
        'proteins_100g': 90,
        'salt_100g': 10,
        'sodium_100g': 4
    }

    # Appliquer les seuils pour chaque nutriment
    for nutrient, threshold in nutrient_thresholds.items():
        if nutrient in df.columns:
            df[nutrient] = df[nutrient].apply(lambda x: x if x <= threshold else float('NaN'))

    return df

In [ ]:
np.set_printoptions(threshold=np.inf)
df = pd.read_csv('/kaggle/input/projet-dataset/data.csv', sep='\t')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

On voit grâce a df.info() que nous avons **106 données quantitatives** et **56 données catégorielles**

**AFFICHAGE DES PROPORTIONS DE VALEURS MANQUANTES**

In [ ]:
df.isna().mean()

**Appel de la fonction de préparation des données**

Notre target sera le Nutriscore qui peut facilement être prédit en fonction des qualités nutritives du produit tels que le sucre, le sel, les acides gras saturés, les protéines, etc...

In [ ]:
target = 'nutrition_grade_fr'
df_cleaned, X, y = automate_data_processing(df, target)
df_cleaned.info()

Il nous reste après le nettoyage des valeurs manquantes **14 données quantitatives** et **21 données catégorielles**. On supprime les features inutiles

In [ ]:
df_cleaned = df_cleaned.drop(columns=['code','url','creator','created_t','created_datetime', 'last_modified_t', 'last_modified_datetime','brands','countries','countries_fr','additives_n','ingredients_from_palm_oil_n', 'states_tags', 'states_fr', 'nutrition-score-fr_100g', 'nutrition-score-uk_100g'])

On vérifie les dimensions de notre ensemble

In [ ]:
df_cleaned.shape

In [ ]:
df_cleaned.info()

In [ ]:
df_cleaned[target].isna().mean()

On supprime les doublons

In [ ]:
df_cleaned = df_cleaned.drop_duplicates()

In [ ]:
features = df_cleaned.columns
#['energy_100g', 'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g', 'sodium_100g']
X = df_cleaned[features]
X.info()

**Premiere Visualisation**

In [ ]:
X.hist(bins=20, figsize=(30, 20))
plt.show()

nutriment_features = [col for col in X.columns if col.endswith('_100g') and col != 'energy_100g']

plt.figure(figsize=(10, 6))
X[nutriment_features].boxplot() 
plt.title("Boxplot des nutriments (100g)")
plt.xticks(rotation=45)
plt.show()

# Figure pour l'énergie
plt.figure(figsize=(6, 6))
X[['energy_100g']].boxplot()
plt.title("Boxplot de l'énergie (kJ pour 100g)")
plt.xticks(rotation=45)
plt.show()

# Figure pour l'énergie
plt.figure(figsize=(6, 6))
X[['ingredients_that_may_be_from_palm_oil_n']].boxplot()
plt.title("Nombre d'ingrédients pouvant contenir de l'huile de palme")
plt.xticks(rotation=45)
plt.show()

On restreint notre projet aux produits vendus en France et dans ses DOM-TOMs

In [ ]:
keywords = [
    'france', 'guadeloupe', 'martinique', 'guyane', 'reunion', 'mayotte', 
    'saint-pierre-et-miquelon', 'saint-barthélemy', 'saint-martin',
    'polynésie française', 'nouvelle-calédonie', 'wallis-et-futuna', 'french-guiana'
]

X = X[X['countries_tags'].str.contains('|'.join(keywords), case=False, na=False)]

X.head()

In [ ]:
X.shape

On supprime dans un premier temps les lignes ayant des valeurs aberrerantes d'un pdv métier

In [ ]:
X = clean_outliers(X)

In [ ]:
palm_oil_5_ingredients = X[X['ingredients_that_may_be_from_palm_oil_n'] >= 4]

print(palm_oil_5_ingredients['main_category'].value_counts())
palm_oil_5_ingredients['main_category'].value_counts().plot(kind='bar')
plt.xlabel("Catégorie de produit")
plt.ylabel("Nombre de produits")
plt.xticks(rotation=45)
plt.title("Répartition des produits avec 5 ingrédients contenant de l'huile de palme")
plt.show()

In [ ]:
energy_threshold = 3500

high_energy_products = X[X['energy_100g'] > energy_threshold]

print(high_energy_products['main_category'].value_counts())

high_energy_products['main_category'].value_counts().plot(kind='bar')
plt.xlabel("Catégorie de produit")
plt.ylabel("Nombre de produits")
plt.title("Répartition des produits avec une énergie > 4000 kJ pour 100g")
plt.show()

In [ ]:
X = clean_nutrient_outliers(X)

In [ ]:
X.hist(bins=20, figsize=(30, 20))
plt.show()

plt.figure(figsize=(10, 6))
X[nutriment_features].boxplot() 
plt.title("Boxplot des nutriments (100g)")
plt.xticks(rotation=45)
plt.show()

# Figure pour l'énergie
plt.figure(figsize=(6, 6))
X[['energy_100g']].boxplot()
plt.title("Boxplot de l'énergie (kJ pour 100g)")
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
X[['salt_100g']].boxplot() 
plt.title("Boxplot du sel (100g)")
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(10, 6))
X[['sodium_100g']].boxplot() 
plt.title("Boxplot du sodium (100g)")
plt.xticks(rotation=45)

In [ ]:
X.isna().mean() * 100